In [1]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime():
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    print("CDT(Current Date and Time):", datetime_string)

print_current_datetime()

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

model.save_pretrained("DialoGPT-medium-by-Microsoft")

print_current_datetime()

model.from_pretrained("DialoGPT-medium-by-Microsoft")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

print_current_datetime()

CDT(Current Date and Time): 2023-03-28 12:42:31.414222
CDT(Current Date and Time): 2023-03-28 12:42:56.851138
>> User:Hi


2023-03-28 12:43:17.231155: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-28 12:43:17.231185: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/gbike/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


DialoGPT: Hi! :D
>> User:Do you speak Korean?
DialoGPT: I do!
>> User:굉장해요.
DialoGPT: I'm Korean, but I don't speak Korean.
>> User:I see.
DialoGPT: I see what you did there.
>> User:What do you mean?
DialoGPT: I see what you did there.
CDT(Current Date and Time): 2023-03-28 12:43:53.472780


In [10]:
# Print the current date and time in the format:
# "YYYY-MM-DD HH:MM:SS.microseconds"
import datetime
def print_current_datetime():
    datetime_string = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")
    print("CDT(Current Date and Time):", datetime_string)

print_current_datetime()

# DialoGPT is based on GPT2
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForCausalLM

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = TFAutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Add a new padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Add a new layer to the model
model.resize_token_embeddings(len(tokenizer))

# Define your training data
train_data = [
    "Hello, how are you?",
    "I am doing well.",
    "That is good to hear.",
    "Thank you.",
]

# Tokenize your training data
train_encodings = tokenizer(train_data, truncation=True, padding=True)

# Define your optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Define your training loop
epochs = 3

print_current_datetime()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    for i in range(len(train_data)):
        input_ids = tf.constant(train_encodings['input_ids'][i])[None, :]
        target_ids = tf.constant(train_encodings['input_ids'][i])[None, :]
        with tf.GradientTape() as tape:
            outputs = model(input_ids)
            logits = outputs.logits
            shift_logits = tf.transpose(logits[:, :-1, :], perm=[0, 2, 1])
            shift_labels = tf.tile(tf.expand_dims(target_ids[:, 1:], axis=-1), [1, 1, 50258])
            shift_labels = tf.tile(shift_labels, [1, 1, 50258])
            loss_value = loss(shift_labels, shift_logits)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        print(f"Batch {i + 1}, loss {loss_value.numpy():.4f}")

print_current_datetime()

# Save the fine-tuned model
model.save_pretrained("DialoGPT-medium-by-Microsoft-finetuned")
tokenizer.save_pretrained("DialoGPT-medium-by-Microsoft-finetuned")

CDT(Current Date and Time): 2023-03-28 14:41:30.824088


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


CDT(Current Date and Time): 2023-03-28 14:41:44.521733
Epoch 1


2023-03-28 14:41:44.994039: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 50517331280 exceeds 10% of free system memory.
2023-03-28 14:41:44.994153: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at tile_ops.cc:199 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[1,5,2525866564] and type int32 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu


ResourceExhaustedError: OOM when allocating tensor with shape[1,5,2525866564] and type int32 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Tile]